In [5]:
import requests
import time
import csv

# Foursquare API Details
API_KEY = "fsq3791aJN4+bpzTQlUFQwCW0IL44wIzjt6/JnPIhhCNwdY="
BASE_URL = "https://api.foursquare.com/v3/places/search"
HEADERS = {
    "Accept": "application/json",
    "Authorization": API_KEY
}

# Define search parameters
CITIES = {
    "Dammam": (26.3927, 50.1183),
    "Khobar": (26.2935, 50.2083),
    "Dhahran": (26.3023, 50.1513)
}
GRID_SPACING = 2.0  # Approx. 2 km spacing
SEARCH_RADIUS = 1500  # 1.5 km radius per request
MAX_RESULTS_PER_AREA = 200  # Adjust to get all businesses

# File output
RESTAURANTS_FILE = "restaurants.csv"
COFFEESHOPS_FILE = "coffeeshops.csv"

# Function to generate grid points
def generate_grid(center_lat, center_lng, spacing, range_km=15):
    """Generates grid points covering an area around a city."""
    points = []
    steps = int((range_km * 2) / spacing)
    
    for i in range(-steps // 2, steps // 2):
        for j in range(-steps // 2, steps // 2):
            lat = center_lat + (i * 0.018)  # Approx. degree change for 2km
            lng = center_lng + (j * 0.018)
            points.append((lat, lng))
    return points

# Function to fetch businesses
def fetch_businesses(lat, lng, category):
    """Fetches businesses from Foursquare API given a location and category."""
    params = {
        "ll": f"{lat},{lng}",
        "radius": SEARCH_RADIUS,
        "limit": 50,  # Foursquare max per request
        "categories": category,
        "fields": "fsq_id,name,geocodes,location,categories,rating,popularity"
    }
    
    all_places = []
    cursor = None

    while len(all_places) < MAX_RESULTS_PER_AREA:
        if cursor:
            params["cursor"] = cursor
        
        response = requests.get(BASE_URL, headers=HEADERS, params=params)
        if response.status_code != 200:
            print(f"Error fetching businesses at ({lat}, {lng}): {response.status_code}")
            break
        
        data = response.json()
        places = data.get("results", [])
        all_places.extend(places)
        
        cursor = data.get("context", {}).get("next")
        if not cursor:
            break  # No more results
        
        time.sleep(1)  # Prevent rate limiting
    
    return all_places[:MAX_RESULTS_PER_AREA]

# Function to save data
def save_to_csv(data, filename):
    """Saves data to CSV file."""
    header = ["Business ID", "Name", "Latitude", "Longitude", "Category", "Rating", "Popularity"]
    with open(filename, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(header)
        writer.writerows(data)

# Main function
def main():
    restaurants_data = []
    coffeeshops_data = []

    for city, (center_lat, center_lng) in CITIES.items():
        print(f"Searching in {city}...")
        grid_points = generate_grid(center_lat, center_lng, GRID_SPACING)

        for lat, lng in grid_points:
            print(f"Fetching data for ({lat}, {lng})...")
            
            # Fetch Restaurants
            restaurants = fetch_businesses(lat, lng, "13065")
            for place in restaurants:
                restaurants_data.append([
                    place.get("fsq_id", "N/A"),
                    place.get("name", "N/A"),
                    place.get("geocodes", {}).get("main", {}).get("latitude", "N/A"),
                    place.get("geocodes", {}).get("main", {}).get("longitude", "N/A"),
                    place.get("categories", [{}])[0].get("name", "N/A"),
                    place.get("rating", "No rating"),
                    place.get("popularity", "No popularity data")
                ])
            
            # Fetch Coffee Shops
            coffeeshops = fetch_businesses(lat, lng, "13032")
            for place in coffeeshops:
                coffeeshops_data.append([
                    place.get("fsq_id", "N/A"),
                    place.get("name", "N/A"),
                    place.get("geocodes", {}).get("main", {}).get("latitude", "N/A"),
                    place.get("geocodes", {}).get("main", {}).get("longitude", "N/A"),
                    place.get("categories", [{}])[0].get("name", "N/A"),
                    place.get("rating", "No rating"),
                    place.get("popularity", "No popularity data")
                ])
            
            time.sleep(1)  # Avoid API rate limit
    
    # Save to CSV
    save_to_csv(restaurants_data, RESTAURANTS_FILE)
    save_to_csv(coffeeshops_data, COFFEESHOPS_FILE)
    print("✅ Data collection complete! Check the CSV files.")

if __name__ == "__main__":
    main()

Searching in Dammam...
Fetching data for (26.248700000000003, 49.9743)...
Fetching data for (26.248700000000003, 49.9923)...
Fetching data for (26.248700000000003, 50.0103)...
Fetching data for (26.248700000000003, 50.028299999999994)...
Fetching data for (26.248700000000003, 50.046299999999995)...
Fetching data for (26.248700000000003, 50.064299999999996)...
Fetching data for (26.248700000000003, 50.0823)...
Fetching data for (26.248700000000003, 50.1003)...
Fetching data for (26.248700000000003, 50.1183)...
Fetching data for (26.248700000000003, 50.1363)...
Fetching data for (26.248700000000003, 50.1543)...
Fetching data for (26.248700000000003, 50.1723)...
Fetching data for (26.248700000000003, 50.1903)...
Fetching data for (26.248700000000003, 50.2083)...
Fetching data for (26.248700000000003, 50.226299999999995)...
Fetching data for (26.2667, 49.9743)...
Fetching data for (26.2667, 49.9923)...
Fetching data for (26.2667, 50.0103)...
Fetching data for (26.2667, 50.028299999999994).